In [ ]:
from __future__ import print_function
import tensorflow.python.keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Input, Reshape
from tensorflow.python.keras.layers import Conv2D, UpSampling2D, BatchNormalization
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.callbacks import EarlyStopping
# from keras import backend as K
import numpy as np
import pickle

In [ ]:
with open('data2.pkl', 'rb') as f:
    x_train = pickle.load(f)
x_train = x_train.astype('float32')
x_train /= 255

In [ ]:
m = 4; n = 1; k = 0;
X_train = np.zeros((x_train.shape[0]-k-m, x_train.shape[1], x_train.shape[2], x_train.shape[3], m));
Y_train = np.zeros((x_train.shape[0]-k-m, x_train.shape[1], x_train.shape[2], x_train.shape[3]));

for i in range(x_train.shape[0]-m-k):
    for j in range(m):
        X_train[i,:,:,:,j] = x_train[i+j]
    Y_train[i] = x_train[i+m]
'''
X_test = np.zeros((k, x_train.shape[1], x_train.shape[2], x_train.shape[3], m));
Y_test = np.zeros((k, x_train.shape[1], x_train.shape[2], x_train.shape[3]));
for i in range(x_train.shape[0]-m-k, x_train.shape[0]-m):
    for j in range(m):
        X_test[i-x_train.shape[0]+m+k,:,:,:,j] = x_train[i+j]
    Y_test[i-x_train.shape[0]+m+k] = x_train[i+m]
    X_test = np.reshape(X_test, [X_test.shape[0], X_test.shape[1], X_test.shape[2], X_test.shape[3]*m])
'''
X_train = np.reshape(X_train, [X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3]*m])
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
Y_train = (Y_train.astype(np.float32) - 127.5) / 127.5

In [ ]:
input_g = Input(shape = (100,))
layerg0 = Dense(3*64*36)(input_g)
layerg0 = Reshape((36,64,3))(layerg0)

layerg1 = Conv2D(64, (3, 3), padding = 'same', activation='relu') (layerg0)
layerg1 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerg1)))
layerg1 = UpSampling2D()(layerg1)

layerg2 = Conv2D(32, (3, 3), padding = 'same', activation='relu') (layerg1)
layerg2 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerg2)))
layerg2 = UpSampling2D()(layerg2)

layerg3 = Conv2D(1, (3, 3), padding = 'same', activation='relu') (layerg2)
layerg3 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerg3)))

output = layerg3

model_g = Model(inputs=input_g, outputs=output)

model_g.summary()

In [ ]:
input_d = Input(shape = (144, 256, 3*(m+1),))
layerd1 = Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', activation='relu') (input_d)
layerd1 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerd1)))

layerd2 = Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', activation='relu') (layerd1)
layerd2 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerd2)))

layerd3 = Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', activation='relu') (layerd2)
layerd3 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerd3)))

layerd4 = Conv2D(3, (3, 3), padding = 'same', activation='relu') (layerd3)
layerd4 = Flatten()(layerd4)
output = Dense(1, activation='sigmoid')(layerd4)
model_d = Model(inputs=input_d, outputs=output);

model_d.summary()

In [2]:
out_g = model_g(input_g)
out = model_d(out_g)
model = Model(inputs=input_g, outputs=out)

NameError: name 'model_g' is not defined

In [3]:
idx = np.random.randint(0, X_train.shape[0], 8)
imgs = np.concatenate((X_train[idx], Y_train[idx]),axis=3);

NameError: name 'np' is not defined

In [4]:
batch_size = 16;
epochs = 10;

half_batch = int(batch_size / 2)

for epoch in range(epochs):
    
    #  Train Discriminator
    # Select a random half batch of images
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = np.concatenate((X_train[idx], Y_train[idx]),axis=3);
    # Sample noise and generate a half batch of new images
    noise = np.random.normal(0, 1, (half_batch, 100))
    gen_imgs = model_g.predict(noise)
    gen_imgs = np.concatenate((X_train[idx], gen_imgs),axis=3);
    # Train the discriminator (real classified as ones and generated as zeros)
    d_loss_real = model_d.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = model_d.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator
    # ---------------------

    # Sample generator input
    noise = np.random.normal(0, 1, (batch_size, 100))

    # Train the generator (wants discriminator to mistake images as real)
    g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))


NameError: name 'np' is not defined

In [5]:
model.compile(loss='binary_crossentropy',
            optimizer='adam', metrics=['accuracy'])

NameError: name 'model' is not defined

In [6]:
stopper = EarlyStopping(monitor='acc',
                              min_delta=0,
                              patience=0,
                              verbose=1, mode='auto')
model.fit(X_train, Y_train, 
          batch_size=8, epochs=10, callbacks=[stopper], verbose=1)

NameError: name 'EarlyStopping' is not defined

In [7]:
Yhat_test = model_d.predict(X_test)

NameError: name 'model_d' is not defined

In [8]:
Y_test.shape

NameError: name 'Y_test' is not defined

In [9]:
import matplotlib.pyplot as plt
plt.imshow(Yhat_test[8,:,:,:])

NameError: name 'Yhat_test' is not defined

batch_size = 8
epochs = 400
half_batch = int(batch_size / 2)

for epoch in range(epochs):
    # ---------------------
    #  Train Discriminator
    # ---------------------
    # Select a random half batch of images
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = X_train[idx]

    # Sample noise and generate a half batch of new images
    noise = np.random.normal(0, 1, (half_batch, 100))
    gen_imgs = model_g.predict(noise)

    # Train the discriminator (real classified as ones and generated as zeros)
    d_loss_real = model_d.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = model_d.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator
    # ---------------------

    # Sample generator input
    noise = np.random.normal(0, 1, (batch_size, 100))

    # Train the generator (wants discriminator to mistake images as real)
    g_loss = model.train_on_batch(noise, np.ones((batch_size, 1)))

    # Plot the progress
    if epoch%10 == 0:
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0]))